In [288]:
using CSV,DataFrames
df = DataFrame(CSV.read("data/gap-development.tsv",delim='\t', allowmissing=:none, normalizenames=true));
first(data,6)

,ID,Text,Pronoun,Pronoun_offset,A,A_offset,A_coref,B,B_offset,B_coref,URL
,String,String,String,Int64,String,Int64,String,String,Int64,String,String
1,development-1,"Zoe Telford -- played the police officer girlfriend of Simon, Maggie. Dumped by Simon in the final episode of series 1, after he slept with Jenny, and is not seen again. Phoebe Thomas played Cheryl Cassidy, Pauline's friend and also a year 11 pupil in Simon's class. Dumped her boyfriend following Simon's advice after he wouldn't have sex with her but later realised this was due to him catching crabs off her friend Pauline.",her,274,Cheryl Cassidy,191,TRUE,Pauline,207,FALSE,http://en.wikipedia.org/wiki/List_of_Teachers_(UK_TV_series)_characters
2,development-2,"He grew up in Evanston, Illinois the second oldest of five children including his brothers, Fred and Gordon and sisters, Marge (Peppy) and Marilyn. His high school days were spent at New Trier High School in Winnetka, Illinois. MacKenzie studied with Bernard Leach from 1949 to 1952. His simple, wheel-thrown functional pottery is heavily influenced by the oriental aesthetic of Shoji Hamada and Kanjiro Kawai.",His,284,MacKenzie,228,TRUE,Bernard Leach,251,FALSE,http://en.wikipedia.org/wiki/Warren_MacKenzie
3,development-3,"He had been reelected to Congress, but resigned in 1990 to accept a post as Ambassador to Brazil. De la Sota again ran for governor of C*rdoba in 1991. Defeated by Governor Angeloz by over 15%, this latter setback was significant because it cost De la Sota much of his support within the Justicialist Party (which was flush with victory in the 1991 mid-terms), leading to President Carlos Menem 's endorsement of a separate party list in C*rdoba for the 1993 mid-term elections, and to De la Sota's failure to regain a seat in Congress.",his,265,Angeloz,173,FALSE,De la Sota,246,TRUE,http://en.wikipedia.org/wiki/Jos%C3%A9_Manuel_de_la_Sota
4,development-4,"The current members of Crime have also performed in San Francisco under the band name ''Remote Viewers``. Strike has published two works of fiction in recent years: Ports of Hell, which is listed in the Rock and Roll Hall of Fame Library, and A Loud Humming Sound Came from Above. Rank has produced numerous films (under his real name, Henry Rosenthal) including the hit The Devil and Daniel Johnston.",his,321,Hell,174,FALSE,Henry Rosenthal,336,TRUE,http://en.wikipedia.org/wiki/Crime_(band)
5,development-5,"Her Santa Fe Opera debut in 2005 was as Nuria in the revised edition of Golijov's Ainadamar. She sang on the subsequent Deutsche Grammophon recording of the opera. For his opera Doctor Atomic, Adams rewrote the role of Kitty Oppenheimer, originally a mezzo-soprano role, for soprano voice, and Rivera sang the rewritten part of Kitty Oppenheimer at Lyric Opera of Chicago, De Nederlandse Opera, and the Metropolitan Opera., all in 2007. She has since sung several parts and roles in John Adams' works, including the soprano part in El Ni*o, and the role of Kumudha in A Flowering Tree in the Peter Sellars production at the New Crowned Hope Festival in Vienna.",She,437,Kitty Oppenheimer,219,FALSE,Rivera,294,TRUE,http://en.wikipedia.org/wiki/Jessica_Rivera
6,development-6,"Sandra Collins is an American DJ. She got her start on the West Coast of the U.S. in Phoenix, Arizona and into residencies in Los Angeles, and eventually moved towards trance. She used American producers to give herself a unique sound. Collins performed for an estimated 80,000 people on the first night of Woodstock '99, and was the first female DJ featured in the Tranceport series of influential recordings. She recently has released two CD mixes under Paul Oakenfold's Perfecto label.",She,411,Collins,236,TRUE,DJ,347,FALSE,http://en.wikipedia.org/wiki/Sandra_Collins


In [289]:
df[:section_min] = min.(df[:Pronoun_offset], df[:A_offset],df[:B_offset])
df[:Pronoun_offset2] = df[:Pronoun_offset] .+ length.(df[:Pronoun])
df[:A_offset2] = df[:A_offset] .+ length.(df[:A])
df[:B_offset2] = df[:B_offset] .+ length.(df[:B])
df[:section_max] = max.(df[:Pronoun_offset2], df[:A_offset2],df[:B_offset2])
df[:Text] = replace.(df[:Text],df[:A].=>"subjecta")
df[:Text] = replace.(df[:Text],df[:B].=>"subjectb")
df[:A_dist] = abs.(df[:Pronoun_offset] .- df[:A_offset])
df[:B_dist] = abs.(df[:Pronoun_offset] .- df[:B_offset]);
train = df[:,[:Pronoun_offset,:A_offset,:B_offset,:section_min,:Pronoun_offset2,:A_offset2,:B_offset2,:section_max,:A_dist,:B_dist]]


,Pronoun_offset,A_offset,B_offset,section_min,Pronoun_offset2,A_offset2,B_offset2,section_max,A_dist,B_dist
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,274,191,207,191,277,205,214,277,83,67
2,284,228,251,228,287,237,264,287,56,33
3,265,173,246,173,268,180,256,268,92,19
4,321,174,336,174,324,178,351,351,147,15
5,437,219,294,219,440,236,300,440,218,143
6,411,236,347,236,414,243,349,414,175,64
7,273,152,253,152,276,161,264,276,121,20
8,337,173,377,173,340,181,393,393,164,40
9,246,255,267,246,249,265,273,273,9,21


In [290]:
train_X = [ [df[i,:Pronoun_offset],df[i,:A_offset],df[i,:B_offset],df[i,:section_min],df[i,:Pronoun_offset2],df[i,:A_offset2],df[i,:B_offset2],df[i,:section_max],df[i,:A_dist],df[i,:B_dist]] for i in 1:size(df)[1] ]

2000-element Array{Array{Int64,1},1}:
 [274, 191, 207, 191, 277, 205, 214, 277, 83, 67]  
 [284, 228, 251, 228, 287, 237, 264, 287, 56, 33]  
 [265, 173, 246, 173, 268, 180, 256, 268, 92, 19]  
 [321, 174, 336, 174, 324, 178, 351, 351, 147, 15] 
 [437, 219, 294, 219, 440, 236, 300, 440, 218, 143]
 [411, 236, 347, 236, 414, 243, 349, 414, 175, 64] 
 [273, 152, 253, 152, 276, 161, 264, 276, 121, 20] 
 [337, 173, 377, 173, 340, 181, 393, 393, 164, 40] 
 [246, 255, 267, 246, 249, 265, 273, 273, 9, 21]   
 [329, 196, 226, 196, 332, 214, 242, 332, 133, 103]
 [300, 168, 212, 168, 303, 184, 223, 303, 132, 88] 
 [304, 217, 285, 217, 307, 224, 302, 307, 87, 19]  
 [293, 247, 273, 247, 296, 255, 289, 296, 46, 20]  
 ⋮                                                 
 [81, 31, 54, 31, 84, 35, 58, 84, 50, 27]          
 [414, 355, 370, 355, 417, 363, 388, 417, 59, 44]  
 [301, 264, 277, 264, 304, 272, 285, 304, 37, 24]  
 [318, 101, 128, 101, 321, 106, 132, 321, 217, 190]
 [316, 343, 372, 316, 319,

In [421]:
labels = DataFrame() 
labels[:A],labels[:B] = df[:A_coref],df[:B_coref];
labels[:Neither] = zeros(2000)
labels[:A] = map(x-> x =="TRUE" ? 1 : 2, labels[:A])
labels[:B] = map(x-> x =="TRUE" ? true : false, labels[:B])
labels[:Neither] = map(x-> x = false, labels[:Neither])

#train_Y = [ [labels[i,:A],labels[i,:B],labels[i,:Neither]] for i in 1:size(labels)[1] ]
train_Y = [ onehot(labels[i,:A], 1:3) for i in 1:size(labels)[1] ]

train_XF = [Float32.(i)./1000 for i in train_X]

2000-element Array{Array{Float32,1},1}:
 [0.274, 0.191, 0.207, 0.191, 0.277, 0.205, 0.214, 0.277, 0.083, 0.067]
 [0.284, 0.228, 0.251, 0.228, 0.287, 0.237, 0.264, 0.287, 0.056, 0.033]
 [0.265, 0.173, 0.246, 0.173, 0.268, 0.18, 0.256, 0.268, 0.092, 0.019] 
 [0.321, 0.174, 0.336, 0.174, 0.324, 0.178, 0.351, 0.351, 0.147, 0.015]
 [0.437, 0.219, 0.294, 0.219, 0.44, 0.236, 0.3, 0.44, 0.218, 0.143]    
 [0.411, 0.236, 0.347, 0.236, 0.414, 0.243, 0.349, 0.414, 0.175, 0.064]
 [0.273, 0.152, 0.253, 0.152, 0.276, 0.161, 0.264, 0.276, 0.121, 0.02] 
 [0.337, 0.173, 0.377, 0.173, 0.34, 0.181, 0.393, 0.393, 0.164, 0.04]  
 [0.246, 0.255, 0.267, 0.246, 0.249, 0.265, 0.273, 0.273, 0.009, 0.021]
 [0.329, 0.196, 0.226, 0.196, 0.332, 0.214, 0.242, 0.332, 0.133, 0.103]
 [0.3, 0.168, 0.212, 0.168, 0.303, 0.184, 0.223, 0.303, 0.132, 0.088]  
 [0.304, 0.217, 0.285, 0.217, 0.307, 0.224, 0.302, 0.307, 0.087, 0.019]
 [0.293, 0.247, 0.273, 0.247, 0.296, 0.255, 0.289, 0.296, 0.046, 0.02] 
 ⋮                      

In [428]:
#Pkg.add("NNlib")
using Flux;
using Flux:crossentropy,throttle
using StatsBase


model2 = Chain(Dense(10,3,relu)
    ,softmax)


    
function loss(x,y)
    x = train_XF
    y = train_Y
    i = sample(1:2000, 1, replace = false)[1]
    l = Flux.mse(model2(x[i]),y[i])
    l
end

loss (generic function with 1 method)

In [429]:
data_XY = zip(train_XF,train_Y)

Base.Iterators.Zip2{Array{Array{Float32,1},1},Array{Flux.OneHotVector,1}}(Array{Float32,1}[[0.274, 0.191, 0.207, 0.191, 0.277, 0.205, 0.214, 0.277, 0.083, 0.067], [0.284, 0.228, 0.251, 0.228, 0.287, 0.237, 0.264, 0.287, 0.056, 0.033], [0.265, 0.173, 0.246, 0.173, 0.268, 0.18, 0.256, 0.268, 0.092, 0.019], [0.321, 0.174, 0.336, 0.174, 0.324, 0.178, 0.351, 0.351, 0.147, 0.015], [0.437, 0.219, 0.294, 0.219, 0.44, 0.236, 0.3, 0.44, 0.218, 0.143], [0.411, 0.236, 0.347, 0.236, 0.414, 0.243, 0.349, 0.414, 0.175, 0.064], [0.273, 0.152, 0.253, 0.152, 0.276, 0.161, 0.264, 0.276, 0.121, 0.02], [0.337, 0.173, 0.377, 0.173, 0.34, 0.181, 0.393, 0.393, 0.164, 0.04], [0.246, 0.255, 0.267, 0.246, 0.249, 0.265, 0.273, 0.273, 0.009, 0.021], [0.329, 0.196, 0.226, 0.196, 0.332, 0.214, 0.242, 0.332, 0.133, 0.103]  …  [0.301, 0.264, 0.277, 0.264, 0.304, 0.272, 0.285, 0.304, 0.037, 0.024], [0.318, 0.101, 0.128, 0.101, 0.321, 0.106, 0.132, 0.321, 0.217, 0.19], [0.316, 0.343, 0.372, 0.316, 0.319, 0.348, 0.376, 0

In [453]:
summary(model2)

"Chain{Tuple{Dense{typeof(relu),TrackedArray{…,Array{Float32,2}},TrackedArray{…,Array{Float32,1}}},typeof(softmax)}}"

In [430]:
evalcb = () -> @show loss(train_XF,train_Y)

#349 (generic function with 1 method)

In [513]:
for i  in 1:100
Flux.train!(loss, params(model2),data_XY, Descent(0.05),cb = throttle(evalcb, 30))
end

loss(train_XF, train_Y) = 0.1335055f0 (tracked)
loss(train_XF, train_Y) = 0.15910918f0 (tracked)
loss(train_XF, train_Y) = 0.14623754f0 (tracked)
loss(train_XF, train_Y) = 0.15240395f0 (tracked)
loss(train_XF, train_Y) = 0.15197676f0 (tracked)
loss(train_XF, train_Y) = 0.17443618f0 (tracked)
loss(train_XF, train_Y) = 0.08732398f0 (tracked)
loss(train_XF, train_Y) = 0.22346441f0 (tracked)
loss(train_XF, train_Y) = 0.2783532f0 (tracked)
loss(train_XF, train_Y) = 0.28170177f0 (tracked)
loss(train_XF, train_Y) = 0.14510942f0 (tracked)
loss(train_XF, train_Y) = 0.09283851f0 (tracked)
loss(train_XF, train_Y) = 0.2688259f0 (tracked)
loss(train_XF, train_Y) = 0.291063f0 (tracked)
loss(train_XF, train_Y) = 0.13353904f0 (tracked)
loss(train_XF, train_Y) = 0.21749124f0 (tracked)
loss(train_XF, train_Y) = 0.12342988f0 (tracked)
loss(train_XF, train_Y) = 0.23255193f0 (tracked)
loss(train_XF, train_Y) = 0.07753532f0 (tracked)
loss(train_XF, train_Y) = 0.25347185f0 (tracked)
loss(train_XF, train_Y) =

In [490]:
model2(train_X[5])

Tracked 3-element Array{Float32,1}:
 0.0f0
 1.0f0
 0.0f0

In [489]:
train_Y[5]

3-element Flux.OneHotVector:
 false
  true
 false

In [409]:
params(model2)

Params([Float32[2.66272 1.38406 … 3.3146 2.88083; 0.111684 -0.502158 … -0.893697 0.411466; … ; 0.537957 -0.414714 … -0.282643 -0.476555; -0.737402 -0.535208 … -0.910332 -0.169445] (tracked), Float32[2.50304, -0.520363, -0.0931673, 0.0, -0.429112] (tracked), Float32[2.17758 1.64441 … -0.744242 0.396827; 1.86046 0.86408 … 0.209833 -0.488413; -2.36168 -2.03426 … -0.305097 -0.373036] (tracked), Float32[1.4051, 2.20709, -1.62545] (tracked)])

In [473]:
using Flux:onecold
accuracy(x, y) = mean(onecold(model2(x), 1:3) .== onecold(y, 1:3))

accuracy (generic function with 1 method)

In [515]:
mean(accuracy.(train_X,train_Y))

0.614

In [509]:
function count(y)
c = 0
for i = 1 : size(y)[1]
if onecold(y[i], 1:3) == 1
    c = c +1
end
    end
 return c   
end


count (generic function with 1 method)

In [510]:
count(train_Y)/2000

0.437